In [1]:
import math

In [2]:
input_file = "07_input.txt"

In [3]:
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size


    def full_path(self):
        if self.folder is None:
            return '/'
        return self.folder.full_path() + self.name
    
    
    def print(self):
        print('-' * self.level + f'{self.name} ({self.size:d})')
    
    
class Dir:
    def __init__(self, name='/', folder=None, level=0):
        self.name = name
        self.files = {}
        self.subfolders = {}
        self.folder = folder
        self.level = level
    
    
    def add_file(self, name, size):
        file = File(name, size)
        self.files[name] = file
        file.folder = self
        file.level = self.level + 1
    
    
    def mkdir(self, name):
        newdir = Dir(name)
        self.subfolders[name] = newdir
        newdir.folder = self
        newdir.level = self.level + 1
        
        
    def ls(self):
        return self.subfolders.keys() + self.files.keys()
    
    
    def compute_size(self):
        self.size = sum(self.subfolders[x].compute_size() for x in self.subfolders)
        self.size += sum(self.files[x].size for x in self.files)
        return self.size
    
    
    def full_path(self):
        if self.folder is None:
            return '/'
        return self.folder.full_path() + self.name + '/'
    
    
    def print(self):
        print('-' * self.level + self.name)
        for c in self.subfolders:
            self.subfolders[c].print()
        for c in self.files:
            self.files[c].print()
    
    
    def sum_smaller(self, size):
        total = sum([self.subfolders[x].sum_smaller(size) for x in self.subfolders])
        if self.size <= size:
            total += self.size
        return total
    
    
    def find_min(self, minsize):
        big_enough = [self.subfolders[x].size for x in self.subfolders if (self.subfolders[x].size >= minsize)]
        if len(big_enough) == 0:
            return math.inf
        return min(big_enough)
    
    def all_sizes(self):
        return [self.size] + sum([self.subfolders[x].all_sizes() for x in self.subfolders], [])


class FileSystem:
    def __init__(self):
        self.root = Dir()
        self.current = self.root


    def cd(self, d):
        if d == '/':
            self.current = self.root
        elif d == '..':
            if self.current == self.root:
                raise ValueError("Cannot go up from root")
            self.current = self.current.folder
        else:
            self.current = self.current.subfolders[d]
    
    
    def mkdir(self, name):
        self.current.mkdir(name)
    
    
    def pwd(self):
        return self.current.full_path()
    
    
    def print(self):
        self.root.print()
    
    
    def parse_command(self, comm):
        if comm[0] == 'cd':
            self.cd(comm[1])
        if comm[0] == 'ls':
            if self.current.subfolders == {} and self.current.files == {}:
                for size, name in comm[1]:
                    if size == 'dir':
                        self.current.mkdir(name)
                    else:
                        self.current.add_file(name, int(size))
            else:
                print(self.current.subfolders.keys(), self.current.files.keys())
                print(comm[1])
                raise ValueError("Directory not empty")
    
    
    def du(self):
        self.root.compute_size()
        
    
    def to_free(self, needed=4*10**7):
        return self.root.size - needed
    
    
    def find_min(self, minsize=None):
        if minsize is None:
            minsize = self.to_free()
        # return min(self.root.size, self.root.find_min(minsize)
        return min([s for s in self.all_sizes() if s >= minsize])
    
    def all_sizes(self):
        return self.root.all_sizes()

In [4]:
with open(input_file) as f:
    terminal = [l.rstrip() for l in f]

In [5]:
commands = []
lsing = False

for x in terminal:
    line = x.split()
    if line[0] == '$':
        if lsing:
            lsing = False
        if line[1] == 'cd':
            commands.append(['cd', line[2]])
        elif line[1] == 'ls':
            lsing = True
            lslist = []
            commands.append(['ls', lslist])
        else:
            raise ValueError(line[1], "is not a command")
    elif lsing:
        lslist.append(line)
    else:
        raise ValueError("string not recognised:", x)

In [6]:
fs = FileSystem()

for command in commands:
    fs.parse_command(command)
    
fs.du()

In [7]:
#part 1
fs.root.sum_smaller(10**5)

1243729

In [8]:
#part 2
fs.find_min()

4443914